# Ghost Demand Detection using Isolation Forest

## 1. Data Loading & Cleaning

In [ ]:
import pandas as pd
df = pd.read_csv("../Data/Amazon Sale Report.csv")

In [ ]:
df.last_valid_index()

In [ ]:
#Checking the data
df.head()

### Dropping the columns that are not required

In [ ]:
# Keep only required columns
df = df[['Date', 'SKU', 'Qty', 'Status']]

In [ ]:
df

In [ ]:
valid_status = ['Shipped', 'Delivered']

df = df[df['Status'].isin(valid_status)]

In [ ]:
df

### Convert order-level data to time-series demand

In [ ]:
df['Date'].head(10)

In [ ]:
df = df.copy()

# Parse Date ONCE, fail fast if anything is wrong
df['Date'] = pd.to_datetime(
    df['Date'].astype(str).str.strip(),
    format='%m-%d-%y',
    errors='raise'
)

In [ ]:
df['Date'].isna().sum()

In [ ]:
df['Date'].dtype

In [ ]:
df['Date'].isna().sum()

In [ ]:
# Status filter first
valid_status = ['Shipped', 'Delivered']
df = df[df['Status'].isin(valid_status)].copy()

In [ ]:
print(df['Date'].dtype)
print(df['Date'].isna().sum())

## 2. Aggregation to Daily SKU-Level Demand


In [ ]:
# Sort for time-series correctness
df = df.sort_values(['SKU', 'Date'])

# Aggregate to daily SKU-level demand
daily_df = (
    df.groupby(['Date', 'SKU'], as_index=False)
      .agg(daily_sales=('Qty', 'sum'))
)

In [ ]:
print("Rows after aggregation:", len(daily_df))
daily_df.head()

In [ ]:
print("Rows before aggregation:", len(df))
print("Rows after aggregation:", len(daily_df))

In [ ]:
df.groupby(['Date', 'SKU']).size().sort_values(ascending=False).head()

## 3. Feature Engineering (Expected Demand)

In [ ]:
#Sorting the data
daily_df = daily_df.sort_values(['SKU', 'Date'])

In [ ]:
daily_df['rolling_mean_7'] = (
    daily_df
    .groupby('SKU')['daily_sales']
    .transform(lambda x: x.rolling(7, min_periods=1).mean())
)

In [ ]:
daily_df['rolling_std_7'] = (
    daily_df
    .groupby('SKU')['daily_sales']
    .transform(lambda x: x.rolling(7, min_periods=1).std())
    .fillna(0)
)

In [ ]:
daily_df['forecast_error'] = (
    daily_df['rolling_mean_7'] - daily_df['daily_sales']
)

In [ ]:
daily_df['demand_change'] = (
    daily_df
    .groupby('SKU')['daily_sales']
    .pct_change()
    .fillna(0)
)

daily_df['volatility_ratio'] = (
    daily_df['rolling_std_7'] / (daily_df['rolling_mean_7'] + 1e-6)
)

In [ ]:
daily_df[['daily_sales', 'rolling_mean_7', 'forecast_error']].describe()

## 4. ML – Isolation Forest Training

In [ ]:
from sklearn.ensemble import IsolationForest

features = [
    'daily_sales',
    'rolling_mean_7',
    'rolling_std_7',
    'forecast_error',
    'demand_change',
    'volatility_ratio'
]

X = daily_df[features]

iso_model = IsolationForest(
    n_estimators=300,
    max_samples=256,
    max_features=0.8,
    contamination=0.02,
    random_state=42
)

iso_model.fit(X)

## 5. Ghost Demand Detection

In [ ]:
daily_df['anomaly_score'] = iso_model.decision_function(X)
daily_df['anomaly_flag'] = iso_model.predict(X)

daily_df['ghost_demand'] = (
    (daily_df['anomaly_flag'] == -1) &
    (daily_df['forecast_error'] > 0)
).astype(int)

## 6. Optimization (OR-Tools)

In [ ]:
from ortools.linear_solver import pywraplp

In [ ]:
ghost_df = daily_df[daily_df['ghost_demand'] == 1].copy()

print(f"Ghost-demand cases to optimize: {len(ghost_df)}")

In [ ]:
# Business assumptions (mock but realistic)
UNIT_PRODUCTION_COST = 50      # cost per unit produced
UNIT_WASTE_PENALTY = 20        # waste / sustainability penalty per unit
MAX_REDUCTION_RATIO = 0.8      # do not cut more than 80% of excess demand


In [ ]:
solver = pywraplp.Solver.CreateSolver('SCIP')

if solver is None:
    raise RuntimeError("Solver not available")

In [ ]:
reduction_vars = {}

for idx, row in ghost_df.iterrows():
    max_cut = max(row['forecast_error'], 0) * MAX_REDUCTION_RATIO
    
    reduction_vars[idx] = solver.NumVar(
        0,
        max_cut,
        f"cut_{idx}"
    )

In [ ]:
# (cost + waste) * production_cut

UNIT_PRODUCTION_COST = 50
UNIT_WASTE_PENALTY = 20
unit_penalty = UNIT_PRODUCTION_COST + UNIT_WASTE_PENALTY


In [ ]:
objective = solver.Objective()

unit_penalty = UNIT_PRODUCTION_COST + UNIT_WASTE_PENALTY

for var in reduction_vars.values():
    objective.SetCoefficient(var, unit_penalty)

objective.SetMaximization()

In [ ]:
status = solver.Solve()

if status != pywraplp.Solver.OPTIMAL:
    raise RuntimeError("Optimization did not find an optimal solution")

print("Optimization solved successfully.")

In [ ]:
status = solver.Solve()

if status != pywraplp.Solver.OPTIMAL:
    raise RuntimeError("Optimization did not find an optimal solution")

print("Optimization solved successfully.")


In [ ]:
ghost_df['recommended_cut'] = 0.0

for idx, var in reduction_vars.items():
    ghost_df.loc[idx, 'recommended_cut'] = var.solution_value()

In [ ]:
ghost_df['cost_saving'] = ghost_df['recommended_cut'] * UNIT_PRODUCTION_COST
ghost_df['waste_reduction_value'] = ghost_df['recommended_cut'] * UNIT_WASTE_PENALTY

In [ ]:
print("Total units reduced:", ghost_df['recommended_cut'].sum())
print("Total production cost saved:", ghost_df['cost_saving'].sum())
print("Total waste reduction value:", ghost_df['waste_reduction_value'].sum())

In [ ]:
ghost_df[[
    'Date',
    'SKU',
    'daily_sales',
    'rolling_mean_7',
    'forecast_error',
    'recommended_cut',
    'cost_saving',
    'waste_reduction_value'
]].sort_values('recommended_cut', ascending=False).head(10)

### Once ghost demand is detected, we use mathematical optimization to compute the optimal production cut for each product-day, constrained by business limits. This allows us to directly quantify cost savings and waste reduction.

In [ ]:
ghost_df['recommended_cut'].describe()

In [ ]:
ghost_df['cost_saving'].sum(), ghost_df['waste_reduction_value'].sum()

### Across ~53,000 daily product observations, our system identified ~467 ghost-demand cases. For these, the optimizer recommends an average production cut of ~2.7 units per case, resulting in approximately ₹63K in cost savings and ₹25K in waste reduction—without impacting normal demand

## 7. Results & Insights

In [ ]:
daily_df['ghost_demand'].value_counts()

In [ ]:
total_rows = len(daily_df)
ghost_rows = daily_df['ghost_demand'].sum()

ghost_pct = (ghost_rows / total_rows) * 100

print(f"Total SKU-days analysed: {total_rows}")
print(f"Ghost demand cases detected: {ghost_rows}")
print(f"Ghost demand rate: {ghost_pct:.2f}%")

In [ ]:
daily_df[daily_df['ghost_demand'] == 1] \
    .sort_values('forecast_error', ascending=False) \
    .head(10)[[
        'Date',
        'SKU',
        'daily_sales',
        'rolling_mean_7',
        'forecast_error'
    ]]

In [ ]:
sku_impact = (
    daily_df[daily_df['ghost_demand'] == 1]
    .groupby('SKU')
    .agg(
        ghost_days=('ghost_demand', 'count'),
        avg_forecast_gap=('forecast_error', 'mean'),
        max_forecast_gap=('forecast_error', 'max')
    )
    .sort_values('ghost_days', ascending=False)
)

sku_impact.head(10)

### What does ghost demand mean in our system?

Ghost demand is detected when:
- Historical data suggests a product should sell well (high rolling average)
- Actual sales fall significantly below expectation
- The pattern is statistically rare compared to normal behavior

These cases indicate inflated demand signals that could lead to overproduction.


In [ ]:
import matplotlib.pyplot as plt

daily_df[daily_df['ghost_demand'] == 1]['forecast_error'].hist(bins=30)
plt.title("Distribution of Forecast Error for Ghost Demand Cases")
plt.xlabel("Expected − Actual Sales")
plt.ylabel("Frequency")
plt.show()

### Out of ~53,000 daily product observations, our system identified ~470 cases of ghost demand less than 1%. These cases are concentrated in a small set of products where expected demand remained high but actual sales dropped, indicating risk of overproduction. We then use optimization to recommend corrective production actions

In [ ]:
# Data for streamlit dashboard
import os

os.makedirs("outputs", exist_ok=True)

final_df = ghost_df[[
    "Date",
    "SKU",
    "daily_sales",
    "rolling_mean_7",
    "forecast_error",
    "ghost_demand",
    "recommended_cut",
    "cost_saving",
    "waste_reduction_value"
]].copy()

final_df.to_csv("outputs/final_results.csv", index=False)


## 8. AI Explainability Using Gemini 

In [ ]:
from google import genai
import os